In [1]:
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta
import math

In [9]:
data_directory = "../Data/"
hololenses = ['A/', 'B/']

In [3]:
def clean_df(df):
    result = df.loc[(df['Movement'] != "start") & (df['EyeTrackingEnabled'] == True) & (df['EyeCalibrationValid'] == True) & (df['EyeTrackingEnabledAndValid'] == True) & (df['EyeTrackingDataValid'] == True) & (df['GazeInputSource'] == "Eyes")].copy()
    result.reset_index(inplace=True, drop=True)
    return result

In [4]:
def calculate_differences(df):
    cleaned_df = clean_df(df)
    difference_df = cleaned_df.loc[1:, ['Movement', 'Time', 'GazeAngleX', 'GazeAngleY']]
    difference_df.reset_index(inplace=True, drop=True)
    difference_df['euclideanDifference'] = difference_df.apply(lambda row: np.linalg.norm(np.array((row['GazeAngleX'], row['GazeAngleY'])) - np.array((cleaned_df.loc[row.name, 'GazeAngleX'], cleaned_df.loc[row.name, 'GazeAngleY']))), axis=1)
    #difference_df['temporalDifference'] = difference_df.apply(lambda row: (datetime.strptime(row['Time'], "%m/%d/%Y %H:%M:%S.%f") - datetime.strptime(cleaned_df.loc[row.name, 'Time'], "%m/%d/%Y %H:%M:%S.%f")).total_seconds(), axis=1)
    return difference_df

In [20]:
def calculate_smooth_pursuit_precision():
    subdirectory = "shifted/"
    smooth_pursuit = []
    for hololens in hololenses:
        print(hololens)
        participantList = os.listdir(data_directory + hololens)
        for participant in participantList:
            print(participant)
            if participant ==  '.DS_Store':
                print("skipping calculation")
                continue
            # if os.path.exists(data_directory + hololens + participant + '/precision/' + subdirectory):
            #     print("already calculated")
            #     continue
            participant_sp = []
            fileList = os.listdir(data_directory + hololens + participant + '/' + subdirectory)
            for file in fileList:
                if '.csv' in file and 'ssHeadConstrained' in file:
                    filePath = data_directory + hololens + participant + '/' + subdirectory + file
                    df = pd.read_csv(filePath)
                    difference_df = calculate_differences(df)
                    currentPursuit = []
                    for index, row in difference_df.iterrows():
                        if row['Movement'] == "static":
                            if len(currentPursuit) > 0:
                                #print(len(currentPursuit))
                                currentPursuit = np.array(currentPursuit)
                                participant_sp.append(math.sqrt(np.sum(np.square(currentPursuit)) / currentPursuit.size))
                            currentPursuit = []
                        elif row['Movement'] == "moving":
                            currentPursuit.append(row['euclideanDifference'])
                    # output_directory = data_directory + hololens + participant + '/precision/' + subdirectory
                    # if not os.path.exists(output_directory):
                    #     os.makedirs(output_directory)
                    # output_path = output_directory + file
                    # difference_df.to_csv(output_path, index=False)
        smooth_pursuit.append(sum(participant_sp)/len(participant_sp))
    return smooth_pursuit

In [21]:
result = calculate_smooth_pursuit_precision()
print(sum(result)/len(result))

A/
03
04
32
34
05
02
.DS_Store
skipping calculation
18
27
29
10
19
43
07
36
09
53
37
08
01
06
39
52
46
41
49
47
25
B/
50
35
51
33
.DS_Store
skipping calculation
20
11
16
42
45
28
17
26
21
44
38
31
54
30
48
24
23
15
12
40
13
14
22
0.10281430674217681


In [32]:
def calculate_saccadic_and_fixation_precision():
    subdirectory = "raw/"
    saccade = []
    fixation = []
    for hololens in hololenses:
        print(hololens)
        participantList = os.listdir(data_directory + hololens)
        for participant in participantList:
            print(participant)
            if participant ==  '.DS_Store':
                print("skipping calculation")
                continue
            # if os.path.exists(data_directory + hololens + participant + '/precision/' + subdirectory):
            #     print("already calculated")
            #     continue
            participant_f = []
            participant_s = []
            fileList = os.listdir(data_directory + hololens + participant + '/' + subdirectory)
            for file in fileList:
                if '.csv' in file and 'calibration' in file:
                    filePath = data_directory + hololens + participant + '/' + subdirectory + file
                    df = pd.read_csv(filePath)
                    difference_df = calculate_differences(df)
                    currentFixation = []
                    lastMovement = difference_df.loc[1, 'Movement']
                    for index, row in difference_df.iterrows():
                        if row['Movement'] == lastMovement:
                            currentFixation.append(row['euclideanDifference'])
                        else:
                            currentFixation = np.array(currentFixation)
                            if row['Movement'] == 'static':
                                participant_s.append(math.sqrt(np.sum(np.square(currentFixation)) / currentFixation.size))
                            elif row['Movement'] == 'transition':
                                participant_f.append(math.sqrt(np.sum(np.square(currentFixation)) / currentFixation.size))
                            currentFixation = []
                        lastMovement = row['Movement']
                    # output_directory = data_directory + hololens + participant + '/precision/' + subdirectory
                    # if not os.path.exists(output_directory):
                    #     os.makedirs(output_directory)
                    # output_path = output_directory + file
                    # difference_df.to_csv(output_path, index=False)
        saccade.append(sum(participant_s)/len(participant_s))
        fixation.append(sum(participant_f)/len(participant_f))
    return [saccade, fixation]

In [33]:
result = calculate_saccadic_and_fixation_precision()
print("saccade")
print(sum(result[0])/len(result[0]))
print("fixation")
print(sum(result[1])/len(result[1]))

A/
03
04
32
34
05
02
.DS_Store
skipping calculation
18
27
29
10
19
43
07
36
09
53
37
08
01
06
39
52
46
41
49
47
25
B/
50
35
51
33
.DS_Store
skipping calculation
20
11
16
42
45
28
17
26
21
44
38
31
54
30
48
24
23
15
12
40
13
14
22
saccade
2.2699584687431362
fixation
0.1428289987944861


In [34]:
def calculate_overall_precision():
    subdirectory = "raw/"
    overall = []
    for hololens in hololenses:
        print(hololens)
        participantList = os.listdir(data_directory + hololens)
        for participant in participantList:
            print(participant)
            if participant ==  '.DS_Store':
                print("skipping calculation")
                continue
            # if os.path.exists(data_directory + hololens + participant + '/precision/' + subdirectory):
            #     print("already calculated")
            #     continue
            participant_overall = []
            fileList = os.listdir(data_directory + hololens + participant + '/' + subdirectory)
            for file in fileList:
                if '.csv' in file and ('calibration' in file or 'ssHeadConstrained' in file):
                    filePath = data_directory + hololens + participant + '/' + subdirectory + file
                    df = pd.read_csv(filePath)
                    difference_df = calculate_differences(df)
                    currentFixation = []
                    lastMovement = difference_df.loc[1, 'Movement']
                    for index, row in difference_df.iterrows():
                        if row['Movement'] == lastMovement:
                            currentFixation.append(row['euclideanDifference'])
                        else:
                            currentFixation = np.array(currentFixation)
                            participant_overall.append(math.sqrt(np.sum(np.square(currentFixation)) / currentFixation.size))
                            currentFixation = []
                        lastMovement = row['Movement']
                    # output_directory = data_directory + hololens + participant + '/precision/' + subdirectory
                    # if not os.path.exists(output_directory):
                    #     os.makedirs(output_directory)
                    # output_path = output_directory + file
                    # difference_df.to_csv(output_path, index=False)
        overall.append(sum(participant_overall)/len(participant_overall))
    return [overall]

In [35]:
result = calculate_overall_precision()
print(sum(result[0])/len(result[0]))

A/
03
04
32
34
05
02
.DS_Store
skipping calculation
18
27
29
10
19
43
07
36
09
53
37
08
01
06
39
52
46
41
49
47
25
B/
50
35
51
33
.DS_Store
skipping calculation
20
11
16
42
45
28
17
26
21
44
38
31
54
30
48
24
23
15
12
40
13
14
22


TypeError: unsupported operand type(s) for +: 'int' and 'list'

In [44]:
def calculate_walking_precision():
    subdirectory = "shifted/"
    smooth_pursuit = []
    fixation = []
    for hololens in hololenses:
        print(hololens)
        participantList = os.listdir(data_directory + hololens)
        for participant in participantList:
            print(participant)
            if participant ==  '.DS_Store':
                print("skipping calculation")
                continue
            # if os.path.exists(data_directory + hololens + participant + '/precision/' + subdirectory):
            #     print("already calculated")
            #     continue
            participant_f = []
            participant_sp = []
            fileList = os.listdir(data_directory + hololens + participant + '/' + subdirectory)
            for file in fileList:
                if '.csv' in file and 'ssWalking' in file:
                    filePath = data_directory + hololens + participant + '/' + subdirectory + file
                    df = pd.read_csv(filePath)
                    difference_df = calculate_differences(df)
                    currentFixation = []
                    lastMovement = difference_df.loc[1, 'Movement']
                    for index, row in difference_df.iterrows():
                        if row['Movement'] == lastMovement:
                            currentFixation.append(row['euclideanDifference'])
                        else:
                            currentFixation = np.array(currentFixation)
                            if row['Movement'] == 'static':
                                participant_sp.append(math.sqrt(np.sum(np.square(currentFixation)) / currentFixation.size))
                            elif row['Movement'] == 'moving':
                                participant_f.append(math.sqrt(np.sum(np.square(currentFixation)) / currentFixation.size))
                            else:
                                print(row['Movement'])
                            currentFixation = []
                        lastMovement = row['Movement']
                    # output_directory = data_directory + hololens + participant + '/precision/' + subdirectory
                    # if not os.path.exists(output_directory):
                    #     os.makedirs(output_directory)
                    # output_path = output_directory + file
                    # difference_df.to_csv(output_path, index=False)
        if len(participant_sp) > 0:
            smooth_pursuit.append(sum(participant_sp)/len(participant_sp))
        else:
            print("sp length 0")
        if len(participant_f) > 0:
            fixation.append(sum(participant_f)/len(participant_f))
        else:
            print("fixation length 0")
    return [smooth_pursuit, fixation]

In [45]:
result = calculate_walking_precision()
print("saccade")
print(sum(result[0])/len(result[0]))
print("fixation")
print(sum(result[1])/len(result[1]))

A/
03
04
32
34
05
02
.DS_Store
skipping calculation
18
27
29
10
19
43
07
36
09
53
37
08
01
06
39
52
46
41
49
47
25
B/
50
35
51
33
.DS_Store
skipping calculation
20
11
16
42
45
28
17
26
21
44
38
31
54
30
48
24
23
15
12
40
13
14
22
saccade
13.229278079810843
fixation
8.598094252772963


In [20]:
spatial_rms = []
temporal_precision = []
dropped_samples = 0
total_samples = 0
for hololens in hololenses:
    participantsList = os.listdir(data_directory + hololens)
    for participant in participantsList:
        if not participant == '.DS_Store':
            print(participant)
            filePath = data_directory + hololens + participant + '/precision/raw/'
            fileList = os.listdir(filePath)
            participant_spatial_rms = []
            for file in fileList:
                if not file == '.DS_Store':
                    df = pd.read_csv(filePath + file)
                    dropped_samples = dropped_samples + len(df[df['temporalDifference'] > 0.04995])
                    total_samples = total_samples + df.shape[0]
                    fixed_df = df[df['Movement'] == 'static']
                    lastXValue = None
                    lastYValue = None
                    fixation_samples = []
                    for index, row in fixed_df.iterrows():
                        if lastXValue == None or lastYValue == None:
                            lastXValue = row['GazeAngleX']
                            lastYValue = row['GazeAngleY']
                        if abs(row['GazeAngleX'] - lastXValue) > 3 and abs(row['GazeAngleY'] - lastYValue) > 3:
                            fixation_samples = np.array(fixation_samples)
                            participant_spatial_rms.append(math.sqrt(np.sum(np.square(fixation_samples)) / fixation_samples.size))
                            fixation_samples = []
                        fixation_samples.append(row['euclideanDifference'])
                        lastXValue = row['GazeAngleX']
                        lastYValue = row['GazeAngleY']
                    #fixed_df['squared'] = fixed_df['euclideanDifference'] ** 2
                    #spatial_rms.append(math.sqrt(fixed_df['squared'].sum() / fixed_df.shape[0]))
                    temporal_precision.append(fixed_df['temporalDifference'].mean())
            #print(len(participant_spatial_rms))
            participant_spatial_rms = np.array(participant_spatial_rms)
            spatial_rms.append(np.mean(participant_spatial_rms))
spatial_rms = np.array(spatial_rms)
temporal_precision = np.array(temporal_precision)
print("Spatial rms: " + str(np.mean(spatial_rms)))
print("Temporal precision: " + str(np.mean(temporal_precision)))
print("Dropped samples: " + str(dropped_samples))
print("Total samples: " + str(total_samples))
print("Percentage dropped: " + str(dropped_samples / total_samples))


03
04
32
34
05
02
18
27
29
10
19
43
07
36
09
37
08
01
06
39
46
41
49
47
25
50
35
33
20
11
16
42
45
28
17
26
21
44
38
31
30
48
24
23
15
12
40
13
14
22
Spatial rms: 6.5306778698533545
Temporal precision: 0.016690032176517378
Dropped samples: 91
Total samples: 2473888
Percentage dropped: 3.6784203650286514e-05
